In [1]:
import numpy as np

In [ ]:
def ReLu(x):
    if (x < 0):
        return 0
    return x

def softmax(x):
    sum = 0
    for i in range(len(x)):
        sum += np.exp(x[i])
    return np.exp(x) / sum

def Max3x3(M):
    max = M[0][0]
    for i in range(3):
        for j in range(3):
            if M[i][j] > max:
                max = M[i][j]
    return max

def MaxPool(M):
    x = len(M[0])
    y = len(M)
    i = 0
    j = 0
    output_x = 0
    output_y = 0
    output_height = (y - 3) // 2 + 1
    output_width = (x - 3) // 2 + 1
    output = np.zeros((output_height, output_width))
    while i <= (y - 3):
        j = 0
        output_x = 0
        while j <= (x - 3):
            print(f'i = {i}')
            print(f'j = {j}')
            print(f'output_x = {output_x}')
            print(f'output_y = {output_y}')
            output[output_y][output_x] = Max3x3(M[i:i+3, j:j+3])
            j += 2
            output_x += 1
        i += 2
        output_y += 1
    return output

def reshape(M):
    output = np.array([])

In [147]:
a = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

In [148]:
reshape(a)

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])